In [1]:
import json
import glob
import pandas as pd
import numpy as np
from sklearnex import patch_sklearn
patch_sklearn()
import sklearn
from numpy import asarray
from PIL import Image
import torch
from torchvision.transforms import ToTensor
from torch import flatten
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

data = '/Users/paddy/Desktop/AiCore/facebook_ml/final_dataset/image_data.json'
images = '/Users/paddy/Desktop/AiCore/facebook_ml/images_raw'
images_corrected = '/Users/paddy/Desktop/AiCore/facebook_ml/Images'

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
image_list = glob.glob(f'{images}/*.jpg')
df = pd.read_json(data)
# df = df.drop(['Unnamed: 0', 'bucket_link', 'image_ref', 'create_time'], axis=1)

df

,image_id,id2,bucket_link,image_ref,create_time,price,category_Baby & Kids Stuff,"category_Clothes, Footwear & Accessories",category_Computers & Software,category_DIY Tools & Materials,category_Health & Beauty,category_Home & Garden,"category_Music, Films, Books & Games",category_Office Furniture & Equipment,category_Other Goods,"category_Phones, Mobile Phones & Telecoms","category_Sports, Leisure & Travel",category_Video Games & Consoles,id
0,912bb259-3ad9-457b-9db1-ce1da9016057,5f5f57d7-778f-4336-bb10-b43863418c8c,https://aicore-product-images.s3.amazonaws.com...,koQAAOSwhZRiE5dD,2022-02-26,5.00,0,0,0,0,0,1,0,0,0,0,0,0,243809c0-9cfc-4486-ad12-3b7a16605ba9
1,b166d305-b852-4bdd-83f4-465b20da94fa,5f5f57d7-778f-4336-bb10-b43863418c8c,https://aicore-product-images.s3.amazonaws.com...,9vgAAOSw8sRiE5dE,2022-02-26,5.00,0,0,0,0,0,1,0,0,0,0,0,0,dcbe214e-cb4b-4f18-ab16-218ec12ab68b
2,68f5a29d-0075-4d60-81c1-ab684a82e50c,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,https://aicore-product-images.s3.amazonaws.com...,FlwAAOSwCkpiFNlV,2022-02-26,5.00,0,0,0,0,0,1,0,0,0,0,0,0,6f228cd8-a2d6-4f23-858b-4a77a2f42d7c
3,f6a309d7-d247-446a-9b5e-aceefdd4334d,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,https://aicore-product-images.s3.amazonaws.com...,4gIAAOSw4ORiFNlZ,2022-02-26,5.00,0,0,0,0,0,1,0,0,0,0,0,0,2275f5f2-ee51-410c-8d85-e5923285a864
4,2c2b3a6f-15b3-4289-937a-15482d9f5781,8292aa4e-7f1b-4655-bf0e-f1f2c9e3ffaf,https://aicore-product-images.s3.amazonaws.com...,sHoAAOSwQoRiD7vf,2022-02-26,5.00,0,0,0,0,0,1,0,0,0,0,0,0,c0e5ef79-09c0-4cb7-9b7c-edd3d53aa530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,ced20b15-f71f-4f18-8b42-8bec96e177c6,a7594f7c-cfd9-49b4-b504-76181fbad85c,https://aicore-product-images.s3.amazonaws.com...,Q3AAAOSw4dZiGUa9,2022-02-27,55.50,0,0,0,0,0,0,0,0,0,0,0,1,a9e31ee1-c706-42a9-97d9-4ca257e4c6f3
5763,ea77b2e0-498b-4015-8884-edd7780c765a,a7594f7c-cfd9-49b4-b504-76181fbad85c,https://aicore-product-images.s3.amazonaws.com...,FWwAAOSwxKhiGUa~,2022-02-27,88.00,0,0,0,0,0,0,0,0,0,0,0,1,f3bcf410-3972-4021-bc0f-dae1892f8793
5764,0f77f8cc-e8bf-4f7d-89ea-62a92d6dbc3f,37a7232e-3579-47f3-8239-b82f516f5473,https://aicore-product-images.s3.amazonaws.com...,i~UAAOSwVj5iGUkx,2022-02-27,63.00,0,0,0,0,0,0,0,0,0,0,0,1,dd1a90ff-4a1f-4ff8-8183-381d79447bf2
5765,89103db6-f8b4-41bd-b03d-8e56a6b72fa9,37a7232e-3579-47f3-8239-b82f516f5473,https://aicore-product-images.s3.amazonaws.com...,puAAAOSw791iGUkz,2022-02-27,179.99,0,0,0,0,0,0,0,0,0,0,0,1,ebe8d613-4aa0-477c-8f1a-256ff8d4f248


In [6]:
df_for_model = df.drop(['bucket_link', 'image_ref', 'create_time','image_ref', 'id', 'id2','price'], axis=1)
# df_for_model = df.drop(['uuid', 'price', 'id', 'id2'], axis=1)
df_for_model

,image_id,category_Baby & Kids Stuff,"category_Clothes, Footwear & Accessories",category_Computers & Software,category_DIY Tools & Materials,category_Health & Beauty,category_Home & Garden,"category_Music, Films, Books & Games",category_Office Furniture & Equipment,category_Other Goods,"category_Phones, Mobile Phones & Telecoms","category_Sports, Leisure & Travel",category_Video Games & Consoles
0,912bb259-3ad9-457b-9db1-ce1da9016057,0,0,0,0,0,1,0,0,0,0,0,0
1,b166d305-b852-4bdd-83f4-465b20da94fa,0,0,0,0,0,1,0,0,0,0,0,0
2,68f5a29d-0075-4d60-81c1-ab684a82e50c,0,0,0,0,0,1,0,0,0,0,0,0
3,f6a309d7-d247-446a-9b5e-aceefdd4334d,0,0,0,0,0,1,0,0,0,0,0,0
4,2c2b3a6f-15b3-4289-937a-15482d9f5781,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,ced20b15-f71f-4f18-8b42-8bec96e177c6,0,0,0,0,0,0,0,0,0,0,0,1
5763,ea77b2e0-498b-4015-8884-edd7780c765a,0,0,0,0,0,0,0,0,0,0,0,1
5764,0f77f8cc-e8bf-4f7d-89ea-62a92d6dbc3f,0,0,0,0,0,0,0,0,0,0,0,1
5765,89103db6-f8b4-41bd-b03d-8e56a6b72fa9,0,0,0,0,0,0,0,0,0,0,0,1


In [7]:
# df.columns
column_indicies = [1,2,3,4,5,6,7,8,9,10,11,12]
new_column = ['0','1','2','3','4','5','6','7','8','9','10','11']
old_names = df_for_model.columns[column_indicies]
df_for_model.rename(columns=dict(zip(old_names, new_column)),inplace=True)
df_for_model['image_id'] =df_for_model['image_id'].astype(str)
# df_for_model = df_for_model.drop(['id'], axis=1)

df_for_model

,image_id,0,1,2,3,4,5,6,7,8,9,10,11
0,912bb259-3ad9-457b-9db1-ce1da9016057,0,0,0,0,0,1,0,0,0,0,0,0
1,b166d305-b852-4bdd-83f4-465b20da94fa,0,0,0,0,0,1,0,0,0,0,0,0
2,68f5a29d-0075-4d60-81c1-ab684a82e50c,0,0,0,0,0,1,0,0,0,0,0,0
3,f6a309d7-d247-446a-9b5e-aceefdd4334d,0,0,0,0,0,1,0,0,0,0,0,0
4,2c2b3a6f-15b3-4289-937a-15482d9f5781,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,ced20b15-f71f-4f18-8b42-8bec96e177c6,0,0,0,0,0,0,0,0,0,0,0,1
5763,ea77b2e0-498b-4015-8884-edd7780c765a,0,0,0,0,0,0,0,0,0,0,0,1
5764,0f77f8cc-e8bf-4f7d-89ea-62a92d6dbc3f,0,0,0,0,0,0,0,0,0,0,0,1
5765,89103db6-f8b4-41bd-b03d-8e56a6b72fa9,0,0,0,0,0,0,0,0,0,0,0,1


In [12]:
# df_for_model['classifiction'] = df_for_model.columns.get_indexer(df.drop('image_id',1).idxmax(1))+1
# df_for_model

/var/folders/1j/z792ghj15kz_y3xd3w6q88d00000gn/T/ipykernel_15063/3020953595.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_for_model['classifiction'] = df_for_model.columns.get_indexer(df.drop('image_id',1).idxmax(1))+1


TypeError: reduction operation 'argmax' not allowed for this dtype

In [11]:
# image_dict = {}
# for i in enumerate(df_for_model[:50]):
#     k = df_for_model['image_id'].astype(str)
#     v = df_for_model.loc[i,:] = 1
# # 

KeyError: "['image_id'] not in index"

In [15]:
# df_pls_work = df_for_model[df_for_model != 0].stack().reset_index(level=0, drop=True)
# print(df_for_model.columns[df_for_model == 1.0])
df_for_model['classification'] = df_for_model.apply(lambda x: x.index[x == 1].tolist(), axis=1)
# df_for_model['classification'] = 
# print(df_for_model)
# df_for_model
df_for_model
# class_list = df_for_model['new'].tolist()


,image_id,0,1,2,3,4,5,6,7,8,9,10,11,classification
0,912bb259-3ad9-457b-9db1-ce1da9016057,0,0,0,0,0,1,0,0,0,0,0,0,[5]
1,b166d305-b852-4bdd-83f4-465b20da94fa,0,0,0,0,0,1,0,0,0,0,0,0,[5]
2,68f5a29d-0075-4d60-81c1-ab684a82e50c,0,0,0,0,0,1,0,0,0,0,0,0,[5]
3,f6a309d7-d247-446a-9b5e-aceefdd4334d,0,0,0,0,0,1,0,0,0,0,0,0,[5]
4,2c2b3a6f-15b3-4289-937a-15482d9f5781,0,0,0,0,0,1,0,0,0,0,0,0,[5]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5762,ced20b15-f71f-4f18-8b42-8bec96e177c6,0,0,0,0,0,0,0,0,0,0,0,1,[11]
5763,ea77b2e0-498b-4015-8884-edd7780c765a,0,0,0,0,0,0,0,0,0,0,0,1,[11]
5764,0f77f8cc-e8bf-4f7d-89ea-62a92d6dbc3f,0,0,0,0,0,0,0,0,0,0,0,1,[11]
5765,89103db6-f8b4-41bd-b03d-8e56a6b72fa9,0,0,0,0,0,0,0,0,0,0,0,1,[11]


In [16]:
dict_model = dict(zip(df_for_model.image_id, df_for_model.classification))
dict_model

{'912bb259-3ad9-457b-9db1-ce1da9016057': ['5'],
 'b166d305-b852-4bdd-83f4-465b20da94fa': ['5'],
 '68f5a29d-0075-4d60-81c1-ab684a82e50c': ['5'],
 'f6a309d7-d247-446a-9b5e-aceefdd4334d': ['5'],
 '2c2b3a6f-15b3-4289-937a-15482d9f5781': ['5'],
 '5e8d17b2-f8e7-403b-bbb1-fa8dfdd3ae6a': ['5'],
 'ed2f3144-6eeb-47a8-855b-76df8774f39b': ['5'],
 '570bbec2-55e6-4e8b-834c-dfa45bea3e94': ['5'],
 '1f99d362-7518-43c9-b50d-c28769512db8': ['5'],
 '30ea4203-cc60-46fd-9554-5e72ad63c30e': ['5'],
 '990e5b83-ceed-4278-8292-3508de709b3f': ['5'],
 '3e700953-0241-4438-b644-402829904239': ['5'],
 'dd1df6e6-aa24-417e-8f39-6a7adb85d501': ['5'],
 '501f151f-067c-4cbb-9ee9-5cce9870f5fa': ['5'],
 '0ba37ad2-caf9-498b-8f60-634c6a05a798': ['5'],
 'ad20ee4a-5318-4bad-8def-69da584a18b3': ['5'],
 '8bf7ff45-0ddd-4911-a947-9f8913e02799': ['5'],
 '94cf20e2-8ee2-4b16-99e5-5a41de67642f': ['5'],
 '15bbc76e-4765-4ca8-9178-2f349458f6dd': ['5'],
 'e47abda6-0e01-4359-97c4-0c15e582ab60': ['5'],
 'b8c0f58f-0e32-4d70-8567-ab89c2725b70':

In [22]:
image_id = dict_model.keys()
image_array = []
image_list_f = glob.glob(f'{images}/*.jpg')
def image_to_tensor():
        for img in image_id:
            image = Image.open(f'{images}/{img}.jpg')
            print(image)
            t = ToTensor()
            tensor = t(image)
            flattened = torch.flatten(tensor)
            flatten_numpy = flattened.numpy()
            print(flattened.dtype)
            image_array.append(flatten_numpy)

            # print(empty)
        return image_array
image_to_tensor()
print(len(image_array))




<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x768 at 0x7FCFD6C6BD30>
torch.float32
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x1024 at 0x7FCFD40C5B50>
torch.float32
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x1024 at 0x7FCFDB22FFA0>
torch.float32
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=768x1024 at 0x7FCFD40C5370>
torch.float32
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=535x390 at 0x7FCFD40E7AF0>
torch.float32
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=535x390 at 0x7FCFD40C5370>
torch.float32
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x768 at 0x7FCFD6C6BC10>
torch.float32
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x768 at 0x7FCFDB22FFA0>
torch.float32
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x680 at 0x7FCFD40C5370>
torch.float32
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x684 at 0x7FCFD40C5B50>
torch.float32
<PIL.JpegImagePlugin.JpegImageFi

KeyboardInterrupt: 

In [ ]:
flat_list = []
for sublist in class_list:
    for item in sublist:
        flat_list.append(item)
print(len(flat_list))

final_cat_list = list(map(int, flat_list))
print(len(final_cat_list))

In [ ]:
class image_setup:
    def __init__(self, df: pd):
        self.df = df
        self.image_list_fixed = glob.glob(f'{images_corrected}/*.jpg')
        self.image_id_list = []
        self.class_list = []
        self.flattened = []
        self.list_of_tuples = []
        self.df_cleaned(df)
        self.classification_names_list(df)
        self.image_correct_prop()
        self.image_to_tensor()
        self.training_tuples()
        self.model_setup()

    
    def df_cleaned(self, df: pd):
        self.df = self.df.drop(['Unnamed: 0', 'bucket_link', 'image_ref', 'create_time'], axis=1)
        self.df = self.df.drop(['uuid', 'price', 'id.1'], axis=1)
        column_indicies = [1,2,3,4,5,6,7,8,9,10,11,12]
        classification_columns = ['0','1','2','3','4','5','6','7','8','9','10','11']
        old_names = self.df.columns[column_indicies]
        self.df.rename(columns=dict(zip(old_names, classification_columns)),inplace=True)
        self.df = self.df.drop(['id'], axis=1)


    def classification_names_list(self, df: pd):
        self.df['new'] = self.df.apply(lambda x: x.index[x == 1].tolist(), axis=1)
        flat_list = []
        class_list = self.df['new'].tolist()
        for sublist in class_list:
            for item in sublist:
                flat_list.append(item)
            self.class_list = list(map(int, flat_list))


    def image_correct_prop(self):
        resized_img = 128
        for n, img in enumerate(self.image_list_fixed):
            image_ite = Image.open(img)
            background = Image.new(mode='RGB', size=(resized_img, resized_img)) 
            original_img = image_ite.size
            max_dim = max(image_ite.size)
            ratio = resized_img / max_dim
            img_corr_ratio =(int(original_img[0] * ratio), int(original_img[1] * ratio))
            img_corr = image_ite.resize(img_corr_ratio)
            # print(img_corr)
            background.paste(img_corr, (((resized_img - img_corr_ratio[0]) // 2), ((resized_img - img_corr_ratio[1]) // 2)))
            # print(background)
            background.save(f'{images_corrected}/{img[50:]}')

    
    def image_to_tensor(self):
        for i, img in enumerate(self.image_list_fixed):
            image = Image.open(img)
            t = ToTensor()
            tensor = t(image)
            flattened = torch.flatten(tensor)
            flatten_numpy = flattened.numpy()
            # print(self.flattened)
            self.flattened.append(flatten_numpy)

        return self.flattened

    def training_tuples(self):
        for i in range(len(self.class_list)):
                # print(image_array[i])
                # print(image_array[i])
                ready_tuple = (self.flattened[i], self.class_list[i])
                self.list_of_tuples.append(ready_tuple)
                # print(self.list_of_tuples)
        return self.list_of_tuples

    def model_setup(self):
        m = len(self.list_of_tuples)
        # sh = image_array[0].shape
        array_size = 49152
        # list_of_tuples
        # print(len(self.list_of_tuples))
        X = np.zeros((m, array_size))
        y = np.zeros(m)
        # print(X.shape, y.shape)
        for item in range(m):
            features, labels = self.list_of_tuples[item]
            # print(features.shape)
            X[item, :] = features
            y[item] = labels
            # print(X)
            # print(y)
        model = LogisticRegression(max_iter= 100)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=59)
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        print ('acc:', accuracy_score(y_test, pred))
        print('prec:', classification_report(y_test, pred))
            
    


            
    
    pass
image_setup(df)

In [ ]:
image_array = []
image_list_f = glob.glob(f'{images_corrected}/*.jpg')
def image_to_tensor():
        for img in image_list_f:
            image = Image.open(img)
            # print(image)
            t = ToTensor()
            tensor = t(image)
            flattened = torch.flatten(tensor)
            flatten_numpy = flattened.numpy()
            # print(flattened.dtype)
            image_array.append(flatten_numpy)

            # print(empty)
        return image_array
image_to_tensor()
print(len(image_array))


In [ ]:
list_of_tuples = []
# final_cat_list

print(len(final_cat_list))
def training_tuples():
        for i in range(6266):
                # print(image_array[i])
                # print(image_array[i])
                ready_tuple = (image_array[i], final_cat_list[i])
                list_of_tuples.append(ready_tuple)
        return list_of_tuples
training_tuples()
print(len(list_of_tuples))
# list_of_tuples[:5].shape

In [ ]:
print(list_of_tuples[1])

In [ ]:
print(image_array[1].shape)
size = 128*128*3
n = len(image_array)
print(len(image_array))
print(image_array[1])


In [ ]:
# m = no of labels
#  features = 49152 = items in list of flattened image :)
m = len(image_setup.list_of_tuples)
sh = image_array[0].shape
array_size = 49152
list_of_tuples
print(len(list_of_tuples))
X = np.zeros((m, array_size))
y = np.zeros(m)
print(X.shape, y.shape)
for item in range(m):
    features, labels = list_of_tuples[item]
    # print(features.shape)
    X[item, :] = features
    y[item] = labels
    # print(X)
    print(y)

In [ ]:
model = LogisticRegression(max_iter= 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=59)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print ('acc:', accuracy_score(y_test, pred))
print('prec:', classification_report(y_test, pred))